<a href="https://colab.research.google.com/github/Latamila/ProjetosComAzure/blob/main/projetoTradutorDIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
PARTE I DO PROJETO

Fazer um modelo para Tradução no Azure AI Services.

In [1]:
!pip install requests python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 15.0 MB/s eta 0:00:00


In [2]:
import requests
from docx import Document
import os

In [3]:
subscription_key = "sua chave"
endpoint = "https://api.cognitive.microsofttranslator.com"
location = 'eastus2'
language_destination = 'pt-br'

def translator_text(text, target_language):
    path = '/translate'
    constructed_url = endpoint + path
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(os.urandom(16))
    }

    body = [{
        'text': text
    }]

    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': target_language
    }
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()

    # Check if the response is a list and contains expected data before accessing it
    if isinstance(response, list) and response and "translations" in response[0]:
        return response[0]["translations"][0]["text"]
    else:
        # Handle the case when the expected data structure is not found
        print(f"Unexpected response format: {response}")  # Print response for debugging
        return "Translation failed"  # Or raise an exception

In [4]:
translator_text("I knew you're somewhere out there, somewhere far away", language_destination)


'Eu sabia que você está em algum lugar lá fora, em algum lugar distante'

In [5]:
def translate_document(path):
    document = Document(path)
    full_text = []
    for paragraph in document.paragraphs:
        translated_text = translator_text(paragraph.text, language_destination)
        full_text.append(translated_text)

    translated_doc = Document()
    for line in full_text:
        print(line)
        translated_doc.add_paragraph(line)

    path_translated = path.replace('.docx', f"{language_destination}.docx")
    translated_doc.save(path_translated)
    return path_translated

In [23]:
input_file = "/content/musica.docx"
translate_document(input_file)

Eu sei que você está em algum lugar lá fora
Em algum lugar distante
Eu quero você de volta
Eu quero você de volta
Meus vizinhos acham que eu sou louco
Mas eles não entendem
Você é tudo que eu tinha
Você é tudo que eu tinha
À noite, quando as estrelas iluminam meu quarto
Eu sento sozinho
Conversando com a Lua
Try'na chegar até você
Na esperança de que você esteja do outro lado
Falando comigo também
Ou eu sou um tolo que se senta sozinho
Falando com a Lua?
Oh, oh
Estou me sentindo como se fosse famoso
O assunto da cidade
Eles dizem que eu enlouqueci
Sim, eu enlouqueci
Mas eles não sabem o que eu sei
Porque quando o sol se põe
Alguém está respondendo
Sim, eles estão respondendo, oh
À noite, quando as estrelas iluminam meu quarto
Eu sento sozinho
Conversando com a Lua
Try'na chegar até você
Na esperança de que você esteja do outro lado
Falando comigo também
Ou eu sou um tolo que se senta sozinho
Falando com a Lua?
Ah, ah, ah
Você já me ouviu chamando?
Oh-oh-oh, oh-oh-oh (ah, ah, ah)
Porque

'/content/musicapt-br.docx'

---
PARTE II DO PROJETO

Fazer um modelo de OPENAI


In [6]:
!pip install requests beautifulsoup4 openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.0 MB/s eta 0:00:00


In [7]:
import requests
from bs4 import BeautifulSoup


def extract_text_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        for script_or_style in soup(["script", "style"]):
            script_or_style.decompose()
        texto = soup.get_text(separator = ' ')

        #limpar texto
        linhas = (line.strip() for line in texto.splitlines())
        parts = (phrase.strip() for line in linhas for phrase in line.split("  "))
        texto_limpo = '\n'.join(part for part in parts if part)
        return texto_limpo
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None


    text = soup.get_text()
    return text

extract_text_from_url("https://aiperspectives.springeropen.com/articles/10.1186/s42467-020-00007-2")

"Can an AI learn political theory? | AI Perspectives & Advances | Full Text\nSkip to main content\nAdvertisement\nSearch\nGet published\nExplore Journals\nBooks\nAbout\nMy account\nSearch all SpringerOpen articles\nSearch\nAI Perspectives & Advances\nAbout\nArticles\nSubmission Guidelines\nSubmit manuscript\nCan an AI learn political theory?\nDownload PDF\nDownload PDF\nCommentary\nOpen access\nPublished:\n07 October 2020\nCan an AI learn political theory?\nStephen J. DeCanio\nORCID:\norcid.org/0000-0002-8734-4695 1 , 2\nAI Perspectives\nvolume \xa02 , Article\xa0number:\xa0 3\n( 2020 )\nCite this article\n4502\nAccesses\n4\nAltmetric\nMetrics\ndetails\nAbstract Alan Turing’s 1950 paper, “Computing Machinery and Intelligence,” contains much more than its proposal of the “Turing Test.” Turing imagined the development of what we today call AI by a process akin to the education of a child. Thus, while Turing anticipated “machine learning,” his prescience brings to the foreground the yet u

In [8]:
from langchain_openai.chat_models.azure import AzureChatOpenAI

In [13]:
client = AzureChatOpenAI(
    azure_endpoint = "seu endpoint",
    api_key = "sua chave",
    api_version = "2023-07-01-preview",
    deployment_name = "gpt-4o-mini",
    max_retries=0
)

def translate_article(text, lang):
    messages = [
        ("system" , "Você atua como tradutor de textos"),
        ("user", f"Traduza o {text} para o idioma {lang} e responda em markdown")
    ]

    response = client.invoke(messages)
    print(response.content)
    return response.content

translate_article("let's see if the deployment was succeeded.", "portugues")

Vamos ver se a implantação foi bem-sucedida.


'Vamos ver se a implantação foi bem-sucedida.'

In [15]:
!pip install tenacity

In [16]:
from tenacity import retry, wait_exponential, stop_after_attempt

# ... (your existing extract_text_from_url function) ...

client = AzureChatOpenAI(
azure_endpoint = "seu endpoint",
    api_key = "sua chave",
    api_version = "2023-07-01-preview",
    deployment_name = "gpt-4o-mini",
    max_retries=0
)

@retry(wait=wait_exponential(multiplier=1, min=4, max=10), stop=stop_after_attempt(6))
def translate_article(text, lang):
    """
    Translates the given text to the target language using Azure OpenAI.

    This function is decorated with `retry` to automatically handle rate limit errors.
    It will retry the request with exponential backoff, waiting for an increasing
    amount of time between attempts (up to 6 attempts).
    """
    messages = [
        ("system" , "Você atua como tradutor de textos"),
        ("user", f"Traduza o {text} para o idioma {lang} e responda em markdown")
    ]

    try:
        response = client.invoke(messages)
        print(response.content)
        return response.content
    except openai.error.RateLimitError as e:
        print(f"Rate limit error: {e}. Retrying...")
        raise  # Re-raise to trigger retry logic


url = 'https://aiperspectives.springeropen.com/articles/10.1186/s42467-020-00007-2'
text = extract_text_from_url(url)
# Reduce the size of text to avoid hitting rate limit
text = text[:500]  # Adjust the slicing to select the desired portion of the text
article = translate_article(text, "pt-br")

print(article)

## Um AI pode aprender teoria política?

**Stephen J. DeCanio**  
ORCID: [orcid.org/0000-0002-8734-4695](orcid.org/0000-0002-8734-4695)

**Publicado em:** 07 de outubro de 2020

### Resumo
O artigo discute a capacidade das inteligências artificiais (IA) em aprender e compreender teorias políticas. A análise envolve a avaliação dos métodos de aprendizado de máquina e sua aplicação em contextos que exigem entendimento de nuances políticas e sociais.

### Palavras-chave
- Inteligência Artificial
- Teoria Política
- Aprendizado de Máquina
- Análise Crítica

### Considerações Finais
O texto conclui que, embora as IAs possam processar grandes volumes de dados e identificar padrões, sua compreensão das complexidades da teoria política ainda é limitada e requer um desenvolvimento mais profundo em termos de interpretação contextual e crítica.

---

Se precisar de mais informações ou detalhes sobre o conteúdo, estou à disposição!
## Um AI pode aprender teoria política?

**Stephen J. DeCanio**  
